#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 mydata and use them to predict the outcome of cases from the last 30 days

In [1]:
! pip install keras-tuner

Defaulting to user installation because normal site-packages is not writeable


In [2]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=22841 sha256=c25d948e9e5cf335c0abcc487abecc60ec758eea3b7dad7dd1d62eab661eb536
  Stored in directory: /tmp/pip-ephem-wheel-cache-gg5_fyjk/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


##Import the Boston311Model class

In [3]:
import os
import pandas as pd
import numpy as np
import pickle
import re
import sys
import time

In [4]:
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree, Boston311KerasNLP


2023-10-09 22:04:01.276330: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 22:04:02.110393: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 22:04:02.117168: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 22:04:13.580392: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


## Load extra features

In [5]:
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

#format tomorrows date as yyyy-mm-dd
tomorrows_date =  now + pd.DateOffset(days=1)
tomorrows_datestring = tomorrows_date.strftime("%Y-%m-%d")

In [6]:
print('today_datestring: ', today_datestring)
print('tomorrows_datestring: ', tomorrows_datestring)

today_datestring:  2023-10-09
tomorrows_datestring:  2023-10-10


In [7]:
#set path to mydata
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_service_id.csv'
json_file = './daily_models/Boston311KerasNLP/20230925_143704_Boston311KerasNLP.json'
model_file = './daily_models/Boston311KerasNLP/20230925_143704_Boston311KerasNLP.h5'
#json_file = './daily_models/Boston311KerasNLP/20231003_031227_Boston311KerasNLP.json'
#model_file = './daily_models/Boston311KerasNLP/20231003_031227_Boston311KerasNLP.h5'
kerasNLP_model = Boston311KerasNLP()
kerasNLP_model.load( json_file, model_file)
kerasNLP_model.predict_date_range['end'] = tomorrows_datestring


In [8]:
data = kerasNLP_model.load_data( 'predict' )
data = kerasNLP_model.enhance_data( data, 'predict')

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:262: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0]


In [9]:
#show first ten records with newest open_dt
data.sort_values(by=['open_dt'], ascending=False).head(10)

,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,closure_reason,case_title,subject,reason,...,location_street_name,location_zipcode,latitude,longitude,geom_4326,source,survival_time,event,ward_number,survival_time_hours
249286,101005100341,2023-10-08 19:57:04,2023-10-11 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,675 E Seventh St,2127.0,42.332209,-71.033120,0101000020E6100000F9A83AA51EC251C091702CD7852A...,Citizens Connect App,NaT,0,7,NaN
249285,101005100340,2023-10-08 19:34:12,2023-10-11 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,28A-28 Thornley St,2125.0,42.314078,-71.059149,0101000020E610000059295519C9C351C0524AE8B33328...,Citizens Connect App,NaT,0,13,NaN
249284,101005100339,2023-10-08 19:29:23,2023-11-07 18:29:27,NaT,ONTIME,Open,,Rodent Activity,Inspectional Services,Environmental Services,...,36 E Springfield St,2118.0,42.336160,-71.075081,0101000020E6100000BE2B951ECEC451C0F6FBA446072B...,Citizens Connect App,NaT,0,8,NaN
249283,101005100338,2023-10-08 19:28:32,2023-10-12 04:30:00,NaT,ONTIME,Open,,Improper Storage of Trash (Barrels),Public Works Department,Code Enforcement,...,2837-2839 Washington St,2119.0,42.321061,-71.093094,0101000020E610000072C92841F5C551C0813E1B831829...,Citizens Connect App,NaT,0,11,NaN
249282,101005100337,2023-10-08 19:08:02,2023-10-11 04:30:00,NaT,ONTIME,Open,,Pick up Dead Animal,Public Works Department,Street Cleaning,...,685-685G E Fifth St,2127.0,42.333766,-71.032519,0101000020E6100000B7D163C914C251C03A031AD7B82A...,Citizens Connect App,NaT,0,6,NaN
249281,101005100336,2023-10-08 19:04:03,2023-10-11 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,10 Alvan Ter,2125.0,42.317160,-71.062951,0101000020E61000006A59BD6107C451C08BC7F6AE9828...,Citizens Connect App,NaT,0,7,NaN
249280,101005100335,2023-10-08 18:49:00,NaN,NaT,ONTIME,Open,,Animal Generic Request,Animal Control,Animal Issues,...,200 Florence St,2131.0,42.286980,-71.120741,0101000020E610000054C66537BAC751C08E9378BFBB24...,Constituent Call,NaT,0,19,NaN
249279,101005100333,2023-10-08 18:44:00,NaN,NaT,ONTIME,Open,,Animal Generic Request,Animal Control,Animal Issues,...,138 Saint Botolph St,2115.0,42.343480,-71.082191,0101000020E6100000228E159C42C551C09F4E5223F72B...,Constituent Call,NaT,0,4,NaN
249278,101005100331,2023-10-08 18:42:00,NaN,2023-10-08 19:00:34,ONTIME,Closed,Case Closed. Closed date : 2023-10-08 23:00:34...,Animal Generic Request,Animal Control,Animal Issues,...,200 Florence St,2131.0,42.286980,-71.120741,0101000020E610000054C66537BAC751C08E9378BFBB24...,Constituent Call,0 days 00:18:34,1,19,0.309444
249277,101005100329,2023-10-08 18:37:00,NaN,NaT,ONTIME,Open,,Schedule Bulk Item Pickup,Public Works Department,Sanitation,...,472 Saratoga St,2128.0,42.380049,-71.027520,0101000020E6100000E8BFA0E4C2C151C0CD681A76A530...,Self Service,NaT,0,1,NaN


In [10]:
clean_data = kerasNLP_model.clean_data_for_prediction( data )

In [11]:
clean_data.head()

,case_enquiry_id,type_Abandoned Bicycle,type_Abandoned Building,type_Abandoned Vehicles,type_Abandoned Vehicles - Private Tow,type_Aircraft Noise Disturbance,type_Alert Boston,type_Animal Found,type_Animal Generic Request,type_Animal Lost,...,ward_number_20,ward_number_21,ward_number_22,ward_number_3,ward_number_4,ward_number_5,ward_number_6,ward_number_7,ward_number_8,ward_number_9
0,101005013792,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
1,101005013793,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,101005013794,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
3,101005013795,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,101005013796,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [12]:
mydata = clean_data

In [13]:
mydata['case_enquiry_id']

0        101005013792
1        101005013793
2        101005013794
3        101005013795
4        101005013796
             ...     
41918    101005100337
41919    101005100338
41920    101005100339
41921    101005100340
41922    101005100341
Name: case_enquiry_id, Length: 41923, dtype: object

In [15]:
import pandas as pd
import numpy as np
from ast import literal_eval
import pickle

pickle_file = 'dataframe.pkl'

X = None

if os.path.exists(pickle_file):
    #check if the file date is earlier than EXTRA_mydata_FILE date
    pickle_file_date = os.path.getmtime(pickle_file)
    EXTRA_mydata_FILE_date = os.path.getmtime(EXTRA_mydata_FILE)
    if pickle_file_date < EXTRA_mydata_FILE_date:
        os.remove(pickle_file)

if os.path.exists(pickle_file):

    X = pickle.load(open(pickle_file, "rb"))
else:
    X = pd.read_csv(EXTRA_mydata_FILE)

    #rename service_request_id to case_enquiry_id
    X.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)
    #remove all rows where case_enquiry_id is non-numeric
    #X = X[X['case_enquiry_id'].str.isnumeric()]
    #convert case_enquiry_id to int64
    #X['case_enquiry_id'] = X['case_enquiry_id'].astype('int64')

    # Convert stringified arrays back to NumPy arrays
    X['cls_embedding'] = X['cls_embedding'].apply(literal_eval).apply(np.array)
    X['pooled_embedding'] = X['pooled_embedding'].apply(literal_eval).apply(np.array)

    pickle.dump(X, open(pickle_file, "wb"))



In [ ]:
#concatenate the two dataframes and reindex
df = X

In [ ]:
df.shape

(1621, 3)

In [ ]:

# Assuming df is your DataFrame and it has columns 'cls_embedding' and 'pooled_embedding'
cls_embedding_flattened = np.stack(df['cls_embedding'].to_numpy())
pooled_embedding_flattened = np.stack(df['pooled_embedding'].to_numpy())

# Remove the old columns
df.drop(['cls_embedding', 'pooled_embedding'], axis=1, inplace=True)

# Add the new flattened columns
df_cls = pd.DataFrame(cls_embedding_flattened, columns=[f'cls_{i}' for i in range(cls_embedding_flattened.shape[1])])
df_pooled = pd.DataFrame(pooled_embedding_flattened, columns=[f'pooled_{i}' for i in range(pooled_embedding_flattened.shape[1])])

df = pd.concat([df, df_cls, df_pooled], axis=1)

In [ ]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype(str)
is_numeric = df['case_enquiry_id'].str.isnumeric()

In [ ]:
df = df[is_numeric]

In [ ]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype('int64')

In [ ]:
df.shape

(1621, 257)

In [ ]:
mydata.shape

(41923, 436)

In [ ]:
#join them so we are left only with records that have mydata in both files
new_mydata = mydata.merge(df, on='case_enquiry_id', how='inner')


In [ ]:
new_mydata.shape

(1563, 692)

In [ ]:

df = new_mydata

In [ ]:
#cast all columns that are type bool to float
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype('float64')
    if df[col].dtype == 'int64':
        df[col] = df[col].astype('float64') 

In [ ]:
#list the number of rows in X and y
print(df.dtypes)


case_enquiry_id                           object
type_Abandoned Bicycle                   float64
type_Abandoned Building                  float64
type_Abandoned Vehicles                  float64
type_Abandoned Vehicles - Private Tow    float64
                                          ...   
pooled_123                               float64
pooled_124                               float64
pooled_125                               float64
pooled_126                               float64
pooled_127                               float64
Length: 692, dtype: object


In [ ]:
#free all unused dataframes
#df_to_delete = [cls_embedding_flattened, pooled_embedding_flattened, df_cls, df_pooled, X, new_mydata, is_numeric, mydata]

#for data_frame in df_to_delete:
#    if data_frame is not None:
#        del data_frame

In [ ]:
case_enquiry_id = df['case_enquiry_id']
X_predict = df.drop(['case_enquiry_id'], axis=1)


In [ ]:

#parse CLS embedding column as array
predictions = kerasNLP_model.model.predict(X_predict)

49/49 [==============================] - 1s 18ms/step


In [ ]:
# Define a function to flatten an array into a string.
def array_to_string(arr):
    return ' '.join(map(str, arr))

# Apply the function along axis 1 (rows).
string_predictions = np.apply_along_axis(array_to_string, axis=1, arr=predictions)

# Now string_predictions is a 1D NumPy array where each element is a string
# that contains all the elements from the corresponding row in the original 2D array.
print(string_predictions)

['0.527308 0.159302 0.15766692 0.050141502 0.03229894 0.020881306 0.03047924 0.02045065 0.0014714698'
 '0.033461705 0.04243029 0.12226363 0.25075707 0.34728995 0.13369562 0.060534194 0.0009273389 0.0086'
 '0.72963375 0.17279217 0.062446713 0.012901192 0.0024996842 0.0010537024 0.0035294055 0.0033019853 '
 ...
 '0.6768266 0.26150236 0.05206993 0.0075110737 0.00077528093 0.00022557205 0.00011467862 0.000185721'
 '0.44518822 0.29983705 0.2267537 0.023201264 0.0022347244 0.0002266277 0.00061335886 0.0019223237 2'
 '0.44518822 0.29983705 0.2267537 0.023201264 0.0022347244 0.0002266277 0.00061335886 0.0019223237 2']


In [ ]:
#combine case_enquiry_id and predictions into a dataframe
predictions_df = pd.DataFrame({'case_enquiry_id':case_enquiry_id, 'prediction':string_predictions})

In [ ]:
bin_labels = [
    "0-12 hours",      # Less than half a day
    "12-24 hours",     # Half to one day
    "1-3 days",        # One to three days
    "4-7 days",        # Four to seven days
    "1-2 weeks",       # One to two weeks
    "2-4 weeks",       # Two to four weeks
    "1-2 months",      # One to two months
    "2-4 months",      # Two to four months
    "4+ months"        # More than four months
]

#predictions_df['prediction'] = predictions_df['prediction'].apply(lambda x: bin_labels[x])

In [ ]:
len(predictions_df)

1563

In [ ]:
print(predictions)

[[5.2730799e-01 1.5930200e-01 1.5766692e-01 ... 3.0479239e-02
  2.0450650e-02 1.4714698e-03]
 [3.3461705e-02 4.2430289e-02 1.2226363e-01 ... 6.0534194e-02
  9.2733890e-04 8.6401822e-03]
 [7.2963375e-01 1.7279217e-01 6.2446713e-02 ... 3.5294055e-03
  3.3019853e-03 1.1841430e-02]
 ...
 [6.7682660e-01 2.6150236e-01 5.2069928e-02 ... 1.1467862e-04
  1.8572135e-04 7.8872993e-04]
 [4.4518822e-01 2.9983705e-01 2.2675370e-01 ... 6.1335886e-04
  1.9223237e-03 2.2558286e-05]
 [4.4518822e-01 2.9983705e-01 2.2675370e-01 ... 6.1335886e-04
  1.9223237e-03 2.2558284e-05]]


In [ ]:
kerasNLP_model.model_type

'Boston311KerasNLP'

In [ ]:
import pandas as pd

#get model_name from json_file name and ml_model_date from json_file name first 8 characters which are YYYYMMDD and change it to YYYY-MM-DD
model_name = json_file.split('/')[-1].split('.')[0]
ml_model_date = model_name[:4] + '-' + model_name[4:6] + '-' + model_name[6:8]

#define an empt pandas dataframe ml_model_df
ml_model_df = pd.DataFrame(columns=['ml_model_name', 'ml_model_type', 'ml_model_date'])

ml_model_df = pd.concat([ml_model_df, pd.DataFrame([{'ml_model_name': model_name, 
                                    'ml_model_type': kerasNLP_model.model_type,
                                    'ml_model_date': ml_model_date}])], ignore_index=True)

print(ml_model_df)

                       ml_model_name      ml_model_type ml_model_date
0  20230925_143704_Boston311KerasNLP  Boston311KerasNLP    2023-09-25


In [ ]:
model_cases = data.drop(['geom_4326','survival_time_hours', 'survival_time', 'event'], axis=1).copy()

In [ ]:
all_model_cases = model_cases 
all_model_predictions = predictions_df

In [ ]:
all_model_predictions['ml_model_name'] = model_name

In [ ]:
all_model_predictions['prediction_date'] = today_datestring

In [ ]:
# %%
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

In [ ]:


# %%
all_model_cases.to_csv(my_datetime+'_311_cases.csv', index=False)


# %%

all_model_predictions.to_csv(my_datetime+'_311_predictions.csv', index=False)

# %%

ml_model_df.to_csv(my_datetime+'_311_ml_models.csv', index=False)

# %%
#create datetime _manifest.txt file with one filename per line
with open(my_datetime+'_manifest.txt', 'w') as f:
    f.write(my_datetime+'_311_cases.csv\n')
    f.write(my_datetime+'_311_predictions.csv\n')
    f.write(my_datetime+'_311_ml_models.csv\n')

# %%
#create an export folder
EXPORT_FOLDER = '~/Documents/BODC-DEI-site/database/seeders'
#copy the csv files to the export folder
!cp {my_datetime}_311_cases.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_predictions.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_ml_models.csv {EXPORT_FOLDER}
!cp {my_datetime}_manifest.txt {EXPORT_FOLDER}



# %% [markdown]
# ** Copy the files to the production server **

# %%
# Define constants for servers
PROD_USER = 'u353344964'
PROD_HOSTNAME = '195.179.236.61'
PORT_NUMBER = 65002
PROD_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel'
STAGE_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel'
PROD_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel/database/seeders'
STAGE_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders'

# %%







In [ ]:
import os


def scp_to_server(filename, user=PROD_USER, hostname=PROD_HOSTNAME, port=PORT_NUMBER, export_folder=PROD_EXPORT_FOLDER):
    """Copy a file to the server using scp."""
    command = f"scp -P {port} {filename} {user}@{hostname}:{export_folder}"
    print(f"Executing: {command}")
    os.system(command)

# Use the function to scp files
files_to_copy = [
    f"{my_datetime}_311_cases.csv",
    f"{my_datetime}_311_predictions.csv",
    f"{my_datetime}_311_ml_models.csv",
    f"{my_datetime}_manifest.txt"
]

# Control where to copy
copy_to_prod = True
copy_to_stage = True

for file in files_to_copy:
    if copy_to_prod:
        scp_to_server(file, export_folder=PROD_EXPORT_FOLDER)
    if copy_to_stage:
        scp_to_server(file, export_folder=STAGE_EXPORT_FOLDER)


# %%
PORT_NUMBER

Executing: scp -P 65002 2023-10-09_19-36-58_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders


Executing: scp -P 65002 2023-10-09_19-36-58_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-09_19-36-58_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-09_19-36-58_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-09_19-36-58_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-09_19-36-58_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-09_19-36-58_manifest.txt u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-09_19-36-58_manifest.txt u353344964@195.179.236.61:/

65002

In [ ]:
# %%
if copy_to_prod:
    !ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {PROD_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'

if copy_to_stage:
    !ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {STAGE_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'


   INFO  Seeding database.  


Manifest files:
/home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2023-10-09_19-36-58_manifest.txt

Total records to process: 43487

Processing /home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2023-10-09_19-36-58_311_cases.csv
100 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 41823.
Total records remaining: 43387.
Time for last 100 records: 0.04 seconds.
Estimated time remaining for this file: 15 seconds.
Estimated time for all files: 16 seconds.
200 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 41723.
Total records remaining: 43287.
Time for last 100 records: 0.01 seconds.
Estimated time remaining for this file: 2 seconds.
Estimated time for all files: 2 seconds.
300 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 41623.
Total records remaining: 43187.
Time for last 100 records: 0.01 seconds.
Estimated time remaining for this file: 2 